In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import numpy as np

In [ ]:
def plot(get_chart, xlabel=None, ylabel=None, title=None , figsize=(10,6), xticks=None):
    titledict = {'fontweight':'bold', 'fontsize': 15}
    labeldict = {'fontsize': 13}
    plt.figure(figsize = figsize)
    ax = get_chart()
    title and ax.set_title(title + "\n", fontdict=titledict)
    xlabel and ax.set_xlabel(xlabel, fontdict=labeldict)
    xlabel and ax.set_ylabel(ylabel, fontdict=labeldict)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#fields = ['competencia','secao','graudeinstrucao','idade','racacor','sexo','tipomovimentacao','salario', 'cbo2002ocupacao', 'categoria']
df = pd.read_csv('../input/d/isabellefo/cageddata/CAGED.csv', delimiter = ";")
#df = pd.read_csv('CAGED2020\CAGED2020.csv', delimiter = ";", low_memory=False)

In [ ]:
df.head()

In [ ]:
df['idade'] = pd.to_numeric(df['idade'], errors='coerce')
df['salario'] = pd.to_numeric(df['idade'], errors='coerce')

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace = True)

In [ ]:
df['descsexo']= df['sexo'].map({1: 'Masculino',3 :'Feminino', 9: 'Nao Identificado'})
df['descraca'] = df['racacor'].map({1: 'Branca',2: 'Preta',3: 'Parda', 4: 'Amarela',5 : 'Indigena',6:'Nao Informada',  9: 'Nao Identificado'})
df['desctipomovimentação'] = df['tipomovimentacao'].map({
10:'Admissão por Primeiro Emprego',
20:'Admissão por Reemprego', 
25:'Admissão por contrato trabalho prazo determinado',
31:'Desligamento por Demissão sem Justa Causa', 
32:'Desligamento por Demissão com Justa Causa', 
33:'Culpa Recíproca',
35:'Admissão por Reintegraçao', 
40:'Desligamento a Pedido',
43:'Término contrato trabalho prazo determinado',
45:'Desligamento por Término de Contrato',
50:'Desligamento por Aposentadoria', 
60:'Desligamento por morte',
70:'Admissão por Transferência', 
80:'Desligamento por Transferência', 
90:'Desliamento por Acordo Empregado e Empregador',
98:'Desligamento de Tipo Ignorado',
99:'Não Identificado'})
df['descgraudeinstrucao'] = df['graudeinstrucao'].map({
        1:"Analfabeto",
        2:"Até 5ª Incompleto",
        3:"5ª Completo Fundamental",
        4:"6ª a 9ª Fundamental",
        5:"Fundamental Completo",
        6:"Médio Incompleto",
        7:"Médio Completo",
        8:"Superior Incompleto",
        9:"Superior Completo",
        10:"Mestrado",
        11:"Doutorado",
        80:"Pós-Graduação completa",
        99:"Não Identificado",
    })
df['desctipodedeficiencia'] = df['tipodedeficiencia'].map({
        1:"Física",
        2:"Auditiva",
        3:"Visual",
        4:"Intelectual (Mental)",
        5:"Múltipla",
        6:"Reabilitado",
        9:"Não Identificado",
        0:"Não Deficiente",
    })

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
conditions = [
    (df['tipomovimentacao'].isin([10,20,25,35,70])),
    (df['tipomovimentacao'].isin([31,32,33,40,43,45,50,60,80,90,98])),
    (df['tipomovimentacao'].isin([99]))
    ]

values = ['Admitido', 'Desligado', 'Não identificado']

df['movimento'] = np.select(conditions, values)

In [ ]:
plot(
    get_chart = lambda: sns.countplot(x='descsexo', hue='movimento',data = df, palette='husl'),
    title = "Contagem de colaboradores admitidos e contratados por sexo",
    xlabel = "Sexo",
    ylabel = "Contagem",
)

In [ ]:
conditions = [
    ((df['idade'] >= 0) & (df['idade'] <= 19)),
    ((df['idade'] >= 20) & (df['idade'] <= 59)),
    (df['idade'] >= 60)
    ]

values = ['Jovem', 'Adulto', 'Idoso']

df['faixa_etaria'] = np.select(conditions, values)

In [ ]:
df[df['faixa_etaria']=='0']['idade']

In [ ]:
def get_chart():
    ax = sns.countplot(x='faixa_etaria',hue='movimento',data = df, palette='husl')
    ax.set(yscale="log")
    return ax
plot(
    get_chart = get_chart,
    xlabel = "Faixa Etária",
    ylabel = "Contagem(log)",
    title = "Contagem de colaboradores admitidos e contratados por faixa etária" 
)

In [ ]:
def get_chart():
    labeled_sectors = df.loc[:, ['movimento', 'graudeinstrucao']]
    labeled_sectors['label'] = df['graudeinstrucao'].map({
        1:"Analfabeto",
        2:"Até 5ª Incompleto",
        3:"5ª Completo Fundamental",
        4:"6ª a 9ª Fundamental",
        5:"Fundamental Completo",
        6:"Médio Incompleto",
        7:"Médio Completo",
        8:"Superior Incompleto",
        9:"Superior Completo",
        10:"Mestrado",
        11:"Doutorado",
        80:"Pós-Graduação completa",
        99:"Não Identificado",
    })
    order = labeled_sectors['label'].value_counts().index
    ax = sns.countplot(y='label', hue='movimento',data=labeled_sectors,order=order, palette='husl')
    return ax

plot(
    get_chart = get_chart,
    title = "Contagem de colaboradores admitidos e contratados por grau instrução",
    ylabel = "Grau de instrução",
    xlabel = "Contagem",
)

In [ ]:
def get_chart():
    labeled_sectors = df.loc[:, ['movimento', 'secao']]
    labeled_sectors['label'] = df['secao'].map({
        "A":"Agricultura, Pecuária, Produção Florestal, Pesca e AqÜIcultura",
        "B":"Indústrias Extrativas",
        "C":"Indústrias de Transformação",
        "D":"Eletricidade e Gás",
        "E":"Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação",
        "F":"Construção",
        "G":"Comércio, Reparação de Veículos Automotores e Motocicletas",
        "H":"Transporte, Armazenagem e Correio",
        "I":"Alojamento e Alimentação",
        "J":"Informação e Comunicação",
        "K":"Atividades Financeiras, de Seguros e Serviços Relacionados",
        "L":"Atividades Imobiliárias",
        "M":"Atividades Profissionais, Científicas e Técnicas",
        "N":"Atividades Administrativas e Serviços Complementares",
        "O":"Administração Pública, Defesa e Seguridade Social",
        "P":"Educação",
        "Q":"Saúde Humana e Serviços Sociais",
        "R":"Artes, Cultura, Esporte e Recreação",
        "S":"Outras Atividades de Serviços",
        "T":"Serviços Domésticos",
        "U":"Organismos Internacionais e Outras Instituições Extraterritoriais",
        "Z":"Não identificado",
    })
    order = labeled_sectors['label'].value_counts().index
    return sns.countplot(y='label', hue='movimento',data=labeled_sectors, order= order, palette='husl')
plot(
    get_chart=get_chart,
    title="Contagem de colaboradores admitidos e contratados por setor",
    ylabel= "Setor",
    xlabel= "Contagem",
    figsize=(10,12)
)

In [ ]:
from IPython.display import Markdown
df_admitidos = df[df['movimento']=='Admitido']
men_mean_income = df_admitidos[df_admitidos['sexo']==1]['salario'].mean()
women_mean_income = df_admitidos[df_admitidos['sexo']==3]['salario'].mean()
gender_diff_income = abs(men_mean_income - women_mean_income)
Markdown("""# Diferença da média salarial entre homens e mulheres admitidos
#### Salário médio de homens admitidos: {0:,.2f}    
#### Salário médio de mulheres admitidas: {1:,.2f}
#### Diferença: {2:,.2f}

""".format(men_mean_income, women_mean_income, gender_diff_income))

In [ ]:
from IPython.display import Markdown
df_admitidos = df[df['movimento']=='Admitido']
men_mean_income = df_admitidos[df_admitidos['sexo']==1]['salario'].median()
women_mean_income = df_admitidos[df_admitidos['sexo']==3]['salario'].median()
gender_diff_income = abs(men_mean_income - women_mean_income)
Markdown("""# Diferença da média salarial entre homens e mulheres admitidos(mediana)
#### Salário médio de homens admitidos: {0:,.2f}    
#### Salário médio de mulheres admitidas: {1:,.2f}
#### Diferença: {2:,.2f}

""".format(men_mean_income, women_mean_income, gender_diff_income))

### Quantidade de admissões/demissões por gênero no período da pandemia

In [ ]:
from IPython.display import Markdown
df_admitidos = df[df['movimento']=='Admitido']
df_desligados = df[df['movimento']=='Desligado']

hired_men = df_admitidos[df_admitidos['sexo']==1]['sexo'].count()
hired_women = df_admitidos[df_admitidos['sexo']==3]['sexo'].count()
hired_gender_diff = abs(hired_men - hired_women)

fired_men = df_desligados[df_desligados['sexo']==1]['sexo'].count()
fired_women = df_desligados[df_desligados['sexo']==3]['sexo'].count()
fired_gender_diff = abs(fired_men - fired_women)

Markdown("""### Quantidade de admissões/demissões por gênero no período da pandemia
| |Homens|Mulheres| Diferença |
|---| ---  |   ---  | --- |
|Adimitidos | {0:,} | {1:,} | {2:,} |
|Deslligados| {3:,} | {4:,} | {5:,} |

""".format(hired_men, hired_women, hired_gender_diff, fired_men, fired_women, fired_gender_diff))

### Diferença salarial entre homens e mulheres desligados

In [ ]:
df_desligados = df[df['movimento']=='Desligado']

fired_men_income = df_desligados[df_desligados['sexo']==1]['salario'].mean()
fired_women_income = df_desligados[df_desligados['sexo']==3]['salario'].mean()
fired_gender_diff = abs(fired_men_income - fired_women_income)

Markdown("""# Diferença salarial entre homens e mulheres desligados
#### Salário médio de homens admitidos: {0:,.2f}    
#### Salário médio de mulheres admitidas: {1:,.2f}
#### Diferença: {2:,.2f}

""".format(fired_men_income, fired_women_income, fired_gender_diff))

In [ ]:
race_mean_income = df.groupby(['racacor'])['salario'].mean()

def get_chart():
    labeled_sectors = df.loc[:, ['movimento', 'racacor']]
    labeled_sectors['label'] = df['racacor'].map({
        1:"Branca",
        2:"Preta",
        3:"Parda",
        4:"Amarela",
        5:"Indígena",
        6:"Não informada",
        9:"Não Identificado"
    })
    order = labeled_sectors['label'].value_counts().index
    ax = sns.barplot(y= race_mean_income.values, x= race_mean_income.index,data =labeled_sectors,  palette='husl')
    return ax
plot(
    get_chart,
    title="Média salárial de moviementação por etinia",
    xlabel="Etinía",
    ylabel="Contagem",
)

### Causas de demissão formalizadas pelos empregadores distribuídas por sexo, raça/cor e faixa etária

In [ ]:
plot(
    lambda: sns.countplot(x='descraca', hue='descgraudeinstrucao',data = df, palette='husl'),
    title="Diferença de escolaridade por raça/cor",
    xlabel="Raça/cor",
    ylabel="Contagem"
)

In [ ]:
plot(
    lambda: sns.countplot(hue='desctipomovimentação', x='descsexo',data = df_desligados, palette='husl'),
    title="Causas de demissão formalizadas pelos empregadores distribuídas por sexo",
    xlabel="Sexo",
    ylabel="Contagem",
)

In [ ]:
plot(
    lambda: sns.countplot(hue='desctipomovimentação', x='descraca',data = df_desligados, palette='husl'),
    title="Causas de demissão formalizadas pelos empregadores distribuídas por raça/cor",
    xlabel="Raça/cor",
    ylabel="Contagem"
)

In [ ]:
plot(
    lambda: sns.countplot(hue='desctipomovimentação', x='faixa_etaria',data = df_desligados, palette='husl'),
    title="Causas de demissão formalizadas pelos empregadores distribuídas por faixa etária",
    xlabel="Faixa Etária",
    ylabel="Contagem",
)

### Duas profissões (ocupações) mais escolhidas por nível escolar

In [ ]:
df['desccbo2002ocupacao'] = df['cbo2002ocupacao'].map({
717020:'Servente de Obras',
622505:'Trabalhador no Cultivo de Arvores Frutiferas', 
514320:'Faxineiro (Desativado em 2010)',
411010:'Assistente Administrativo', 
622110:'Trabalhador da Cultura de Cana-De-Acucar', 
411005:'Auxiliar de Escritorio, em Geral',
234520:'Professor de Ensino Superior na Area de Pratica de Ensino',
234505:'Professor de Ensino Superior na Area de Didatica',
521110:'Vendedor de Comercio Varejista', 
784205:'Alimentador de Linha de Producao',
223505:'Enfermeiro'})

In [ ]:
df.groupby('descgraudeinstrucao')['desccbo2002ocupacao'].value_counts().groupby(level=0, group_keys=False).head(2)

In [ ]:
df['competencia'].value_counts()

In [ ]:
df_admitidos[df_admitidos['tipodedeficiencia']!=0]['desctipodedeficiencia'].value_counts()

In [ ]:
df_desligados[df_desligados['tipodedeficiencia']!=0]['desctipodedeficiencia'].value_counts()

In [ ]:
df_admitidos['faixa_etaria'].value_counts()

In [ ]:
gastos = pd.read_csv('../input/gatos-2020/Gastos_2020_.csv', sep=';')

In [ ]:
gastos.head(13)

In [ ]:
gastos = gastos.drop([gastos.index[12]])

In [ ]:
gastos.info()

In [ ]:
gastos['ICMS'] = gastos['ICMS'].apply(lambda x:x.replace(".", ""))
gastos['ICMS'] = gastos['ICMS'].apply(lambda x:x.replace(",", "."))
gastos['ICMS'] = pd.to_numeric(gastos['ICMS'])


In [ ]:
gastos['Total'] = gastos['Total'].apply(lambda x:x.replace(".", ""))
gastos['Total'] = gastos['Total'].apply(lambda x:x.replace(",", "."))
gastos['Total'] = pd.to_numeric(gastos['Total'])

In [ ]:
plot(
    lambda: sns.barplot(x='Meses', y='Total',data = gastos, palette='husl'),
    title="Repasses de 2020 - São José dos Campos",
    xlabel="Meses",
    figsize=(15, 6),
    ylabel="Total",
)
